# Load Data

In [15]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
import calendar
import scipy.stats as stats
pd.set_option('display.max_columns', None)

In [2]:
sales_df = pd.read_csv('/Users/timlee/Desktop/ds021720/game-sales-FIDS/downloads/games_clean_df.csv')

In [3]:
sales_df.shape

(6242, 16)

In [161]:
np.array(sales_df.Year).mean()

2008.035405318808

In [4]:
print(sum(sales_df['Global_Sales'].notnull()))
print(sum(sales_df['NA_Sales'].notnull()))
print(sum(sales_df['PAL_Sales'].notnull()))
print(sum(sales_df['JP_Sales'].notnull()))
print(sum(sales_df['Other_Sales'].notnull()))

6242
5824
5699
1929
6141


In [ ]:
# PS - PS2 - PS3 - PS4
# XBOX - X360 - XONE
# N64 - GC - WII - WIIU - Switch
# GBA - DS - 3DS
# PSP - VITA
# Orphans: DC, PC

# Add Own Data and Shape Dates

In [169]:
consoles = {
    'N64': {'Company': 'Nintendo', 'Family': 'Nintendo', 'Launch': 285, 'Successor': 'GC', 'Predecessor': 'origin', 'Units Sold': 32.4},
    'GC': {'Company': 'Nintendo', 'Family': 'Nintendo', 'Launch': 222, 'Successor': 'WII', 'Predecessor': 'N64', 'Units Sold': 21.7},
    'WII': {'Company': 'Nintendo', 'Family': 'Nintendo', 'Launch': 160, 'Successor': 'WIIU', 'Predecessor': 'GC', 'Units Sold': 101.6},
    'WIIU': {'Company': 'Nintendo', 'Family': 'Nintendo', 'Launch': 88, 'Successor': 'Switch', 'Predecessor': 'WII', 'Units Sold': 13.6},
    'Switch': {'Company': 'Nintendo', 'Family': 'Nintendo', 'Launch': 36, 'Successor': 'current', 'Predecessor': 'WIIU', 'Units Sold': 52.5},
    'PS': {'Company': 'Sony', 'Family': 'Sony', 'Launch': 303, 'Successor': 'PS2', 'Predecessor': 'origin', 'Units Sold': 102.5},
    'PS2': {'Company': 'Sony', 'Family': 'Sony', 'Launch': 240, 'Successor': 'PS3', 'Predecessor': 'PS', 'Units Sold': 155},
    'PS3': {'Company': 'Sony', 'Family': 'Sony', 'Launch': 160, 'Successor': 'PS4', 'Predecessor': 'PS2', 'Units Sold': 87.4},
    'PS4': {'Company': 'Sony', 'Family': 'Sony', 'Launch': 77, 'Successor': 'PS5', 'Predecessor': 'PS3', 'Units Sold': 108.9},
    'PS5': {'Company': 'Sony', 'Family': 'Sony', 'Launch': -7, 'Successor': 'current', 'Predecessor': 'PS4', 'Units Sold': 'None'},
    'XBOX': {'Company': 'Microsoft', 'Family': 'Microsoft', 'Launch': 220, 'Successor': 'X360', 'Predecessor': 'origin', 'Units Sold': 24},
    'X360': {'Company': 'Microsoft', 'Family': 'Microsoft', 'Launch': 172, 'Successor': 'XONE', 'Predecessor': 'XBOX', 'Units Sold': 84},
    'XONE': {'Company': 'Microsoft', 'Family': 'Microsoft', 'Launch': 77, 'Successor': 'XBS', 'Predecessor': 'X360', 'Units Sold': 46.9},
    'XBS': {'Company': 'Microsoft', 'Family': 'Microsoft', 'Launch': -7, 'Successor': 'current', 'Predecessor': 'XONE', 'Units Sold': 'None'},
    'GBA': {'Company': 'Nintendo', 'Family': 'Nintendo_Handheld', 'Launch': 228, 'Successor': 'DS', 'Predecessor': 'origin', 'Units Sold': 85.1},
    'DS': {'Company': 'Nintendo', 'Family': 'Nintendo_Handheld', 'Launch': 184, 'Successor': '3DS', 'Predecessor': 'GBA', 'Units Sold': 154},
    '3DS': {'Company': 'Nintendo', 'Family': 'Nintendo_Handheld', 'Launch': 109, 'Successor': 'current', 'Predecessor': 'DS', 'Units Sold': 75.7},
    'PSP': {'Company': 'Sony', 'Family': 'Sony_Handheld', 'Launch': 183, 'Successor': 'VITA', 'Predecessor': 'origin', 'Units Sold': 80},
    'VITA': {'Company': 'Sony', 'Family': 'Sony_Handheld', 'Launch': 99, 'Successor': 'current', 'Predecessor': 'PSP', 'Units Sold': 10},
    'DC': {'Company': 'Sega', 'Family': 'Sega', 'Launch': 256, 'Successor': 'current', 'Predecessor': 'origin', 'Units Sold': 9},
    'PC': {'Company': 'PC', 'Family': 'PC', 'Launch': 0, 'Successor': 'current', 'Predecessor': 'origin', 'Units Sold': 100},
}

In [6]:
# Modification for dates and building timeline of "months ago"

# new column for years ago
sales_df['years_ago'] = 2020 - sales_df['date'].str[-4:].astype(int)

# take first 3 characters of date for month abbreviation
sales_df['month_abbrev'] = sales_df['date'].str[:3]


def month_numerize(abbrev):
    for k, v in enumerate(calendar.month_abbr):
        if abbrev == v:
            return k
        
# column for numerical representations of month (Jan is 01, Feb is 02, etc.)
sales_df['month_num'] = sales_df['month_abbrev'].apply(month_numerize)

# months ago from April 2020
sales_df['months_ago'] = (sales_df['years_ago'] * 12) + (12 - sales_df['month_num']) + 4 - 12

# dummy categories for months, haven't put this into the dataframe yet
month_dummies = pd.get_dummies(sales_df['month_abbrev'], drop_first=True)

In [7]:
# return a percentage of when a game was released in a console's lifespan

# GTAV (PS3) was released right before the PS4 launched, so it has a 98% value
# God of War 2 (PS2) was released after the PS3 launched, so it has a value over 100%
# (game_release - console_launch) / (console_launch - successor_launch)
def generation_lifespan(console, months_ago):
    successor = consoles[console]['Successor']
    if successor == 'current':
        return successor
    else:
        successor_launch = consoles[successor]['Launch']
        console_launch = consoles[console]['Launch']
        return abs((months_ago - console_launch) / (console_launch - successor_launch))

In [13]:
# testing of generation_lifespan function
test_console = 'PS4'
test_months_ago = 65
generation_lifespan(test_console, test_months_ago)

0.14285714285714285

In [8]:
# create new column for generation_lifespan
sales_df['generation_lifespan'] = sales_df.apply(lambda x: generation_lifespan(x['Platform'], x['months_ago']), axis=1)

In [22]:
console_family = {
    'Nintendo' : ['N64', 'GC', 'WII', 'WIIU', 'Switch'],
    'Sony' : ['PS', 'PS2', 'PS3', 'PS4'],
    'Microsoft' : ['XBOX', 'X360', 'XONE'],
    'Nintendo_Handheld' : ['GBA', 'DS', '3DS'],
    'Sony_Handheld' : ['PSP', 'VITA'],
    'Sega' : ['DC'],
    'PC' : ['PC']
}

In [12]:
# PS - PS2 - PS3 - PS4
# XBOX - X360 - XONE
# N64 - GC - WII - WIIU - Switch
# GBA - DS - 3DS
# PSP - VITA
# Orphans: DC, PC
sales_df.Platform.value_counts()

PS2       1041
X360       753
PS3        653
XBOX       540
PC         490
PS4        400
WII        394
DS         388
PSP        343
GC         312
XONE       236
GBA        215
3DS        146
Switch      95
VITA        85
PS          78
WIIU        44
N64         17
DC          12
Name: Platform, dtype: int64

In [11]:
# we lack any sort of "epoch", or reference date to start counting months
# so we'll count backwards from April 1 2020

# "months ago" calculation performed at:
# https://www.timeanddate.com/date/durationresult.html
# set "State Date" to the Release date (game) or Launch date (console)
# set "End Date" to April 1 2020
# record ~months ago~, and discard any additional days

# Switch launch in March 2017, 36 months ago
# PS4 launch in November 2013, 77 months ago
# XONE launch in November 2013, 77 months ago
# Wii U launch in November 2012, 88 months ago
# VITA launch in December 2011, 99 months ago
# 3DS launch in February 2011, 109 months ago
# PS3 launch in November 2006, 160 months ago
# Wii launch in November 2006, 160 months ago
# X360 launch in November 2005, 172 months ago
# PSP launch in December 2004, 183 months ago
# NDS launch in November 2004, 184 months ago
# XBOX launch in November 2001, 220 months ago
# GC launch in September 2001, 222 months ago
# GBA launch in March 2001, 228 months ago
# PS2 launch in March 2000, 240 months ago
# DC launch in November 1998, 256 months ago
# N64 launch in June 1996, 285 months ago
# PS1 launch in December 1994, 303 months ago


In [21]:
#testing for months_ago
test_years_ago = 7
test_month = 9
(test_years_ago * 12) + (12 - test_month) + 4 - 12

79

# DataFrame only containing games in a console generation

In [10]:
generation_df = sales_df[sales_df.generation_lifespan != 'current']

In [73]:
generation_df.shape

(5414, 21)

# Hypothesis Testing: Games Released in Relation to a Console's Lifespan

We'll be examining the sales of games and any possible correlation with their corresponding console's lifespan. 
For the following hypothesis testing, we will be using this function to conduct two-sample independent t-tests and print various statistics.

In [149]:
def conduct_2samp_t(experiment, control):
    test_statistic, p_value = stats.ttest_ind(experiment, control)
    print('There are ' + str(len(experiment)) + ' data points in the experimental group.')
    print('There are ' + str(len(control)) + ' data points in the control group.')
    print('The experimental mean is: ' + str(round(experiment.mean(), 4)))
    print('The control mean is: ' + str(round(control.mean(), 4)))
    print('The t-statistic is: ' + str(round(test_statistic.mean(), 4)))
    print('The p-value is: ' + str(round(p_value.mean(), 4)))

We were able to detect a signfigant difference in sales when considering games that were launched on a console after that console's successor had launched. For example, releasing a PS2 game after the PS3 was launched.

We were able to **reject** the null hypothesis and **support** the alternative hypothesis:

**H0**: Releasing a game on a console whose successor has launched does not have an effect on sales

**H1**: Releasing a game on a console whose successor has launched has an effect on sales

In [150]:
print("Results for the t-test on games released for a console after the console's successor was launched")
print("In other words, releasing a game on an obsolete console, like a PS2 game released after the PS3 launched\n")

released_in_next_gen = np.array(generation_df[generation_df.generation_lifespan >= 1].Global_Sales)
control = np.array(generation_df[generation_df.generation_lifespan < 1].Global_Sales)
conduct_2samp_t(released_in_next_gen, control)

Results for the t-test on games released for a console after the console's successor was launched
In other words, releasing a game on an obsolete console, like a PS2 game released after the PS3 launched

There are 331 data points in the experimental group.
There are 5083 data points in the control group.
The experimental mean is: 0.5605
The control mean is: 0.7083
The t-statistic is: -2.0226
The p-value is: 0.0432


## Launch Titles and Swan Songs

We conducted additional testing to seek significant differences in sales among groups of games released near the beginning or end of a console's lifespan, specifically shortly after a console's launch or right before the console's successor's launch.

We were unable to reject the null hypothesis for any of these situations.

**H0**: Releasing a game on a console at the (beginning or end) of a console's lifespan has no effect on sales

**H1**: Releasing a game on a console at the (beginning or end) of a console's lifespan has an effect on sales

### Launch Titles

In [142]:
print("Results for the t-test on games released for a console in the first 10% of the console's lifespan")
print("These games are sometimes called 'launch titles'\n")

released_in_first_10 = np.array(generation_df[(generation_df.generation_lifespan >= 0) & (generation_df.generation_lifespan < .1)].Global_Sales)
control = np.array(generation_df[generation_df.generation_lifespan >= .1].Global_Sales)
conduct_2samp_t(released_in_first_10, control)

Results for the t-test on games released for a console in the first 10% of the console's lifespan
These games are sometimes called 'launch titles'

There are 361 data points in observation
There are 5053 data points in control
The observation mean is: 0.6401
The control mean is: 0.7035
The t-statistic is: -0.9031
The p-value is: 0.3665


In [143]:
print("Results for the t-test on games released for a console in the first 20% of the console's lifespan")
print("These are again launch titles, but with a less-stringent requirement\n")

released_in_first_20 = np.array(generation_df[(generation_df.generation_lifespan > 0) & (generation_df.generation_lifespan < .2)].Global_Sales)
control = np.array(generation_df[generation_df.generation_lifespan >= .2].Global_Sales)

conduct_2samp_t(released_in_first_20, control)

Results for the t-test on games released for a console in the first 20% of the console's lifespan
These are again launch titles, but with a less-stringent requirement

There are 805 data points in observation
There are 4575 data points in control
The observation mean is: 0.6851
The control mean is: 0.6992
The t-statistic is: -0.2868
The p-value is: 0.7743


### Swan Songs

In [156]:
print("Results for the t-test on games released for a console in the final 10% of the console's lifespan")
print("In simpler terms, a game released on a console right before the console's successor was launched\n")
released_in_final_90 = np.array(generation_df[(generation_df.generation_lifespan >= .9) & (generation_df.generation_lifespan < 1)].Global_Sales)
control = np.array(generation_df[(generation_df.generation_lifespan < .9) | (generation_df.generation_lifespan >= 1)].Global_Sales)
conduct_2samp_t(released_in_final_90, control)

Results for the t-test on games released for a console in the final 10% of the console's lifespan
In simpler terms, a game released on a console right before the console's successor was launched

There are 288 data points in the experimental group.
There are 5126 data points in the control group.
The experimental mean is: 0.7899
The control mean is: 0.6942
The t-statistic is: 1.2274
The p-value is: 0.2197


In [157]:
print("Results for the t-test on games released for a console in the final 20% of the console's lifespan")
print("Again, this deals with a game released on a console right before the console's successor was launched\n")
released_in_final_80 = np.array(generation_df[(generation_df.generation_lifespan >= .8) & (generation_df.generation_lifespan < 1)].Global_Sales)
control = np.array(generation_df[(generation_df.generation_lifespan < .8) | (generation_df.generation_lifespan >= 1)].Global_Sales)

conduct_2samp_t(released_in_final_80, control)

Results for the t-test on games released for a console in the final 20% of the console's lifespan
Again, this deals with a game released on a console right before the console's successor was launched

There are 651 data points in the experimental group.
There are 4763 data points in the control group.
The experimental mean is: 0.7562
The control mean is: 0.6915
The t-statistic is: 1.2027
The p-value is: 0.2291


# Hypothesis Testing: Games Re-released on a Successor Platform

When examining games that were released toward the end of a console's lifespan, we found many games were launched near the end of a console's lifespan, then re-released on the console's successor.

We decided to track these 131 games' sales on the predecessor platform and the successor platform. With two data points per re-released game, we're able to conduct a two sample paired t-test.

We were able to **reject** the null hypothesis and **support** the alternative hypothesis:

**H0**: A game released on two consoles, one a predecessor and one a successor, will have equal sales for each release

**H1**: A game released on two consoles, one a predecessor and one a successor, will NOT have equal sales for each release

In [155]:
predecessor_sales = []
successor_sales = []
rereleased_games = []

for index, row in sales_df.iterrows():
    game = row['Name']
    platform = row['Platform']

    successor = consoles[platform]['Successor']
    if successor !='current':
        predecessor_sale = (sales_df[(sales_df.Name == game) & (sales_df.Platform == platform)]).iloc[0]['Global_Sales']
        try:
            successor_sale = (sales_df[(sales_df.Name == game) & (sales_df.Platform == successor)]).iloc[0]['Global_Sales']
            predecessor_sales.append(predecessor_sale)
            successor_sales.append(successor_sale)
            rereleased_games.append(game)
        except IndexError:
            pass

predecessor_sales = np.array(predecessor_sales)
successor_sales = np.array(successor_sales)

In [121]:
test_statistic, p_value = stats.ttest_rel(predecessor_sales, successor_sales)
print('There are ' + str(len(predecessor_sales)) + ' data points in predecessor.')
print('There are ' + str(len(successor_sales)) + ' data points in successor.')
print('The mean of predecessor sales is: ' + str(round(predecessor_sales.mean(), 3)))
print('The mean of successor sales is: ' + str(round(successor_sales.mean(), 3)))
print('The t-statistic is: ' + str(round(test_statistic.mean(), 3)))
print('The p-value is: ' + str(round(p_value.mean(), 5)))

There are 131 data points in predecessor.
There are 131 data points in successor.
The mean of predecessor sales is: 1.533
The mean of successor sales is: 1.005
The t-statistic is: 3.874
The p-value is: 0.00017


# Creation of column for rereleases

In [164]:
    #small dataframe for testing
    small_sales_df = sales_df.head(30)

In [178]:
# function to find games that are rereleases, returning 1 if it's a predecessor, 2 if it's a successor, and 0 if it's neither (a game that wasn't rereleased)
def find_rereleases(game, platform, dataframe):
    predecessor = consoles[platform]['Predecessor']
    successor = consoles[platform]['Successor']
    if successor !='current':
        try:
            predecessor = (dataframe[(dataframe.Name == game) & (dataframe.Platform == predecessor)]).iloc[0]['Global_Sales']
            return 2
        except IndexError:
            try:
                successor = (dataframe[(dataframe.Name == game) & (dataframe.Platform == successor)]).iloc[0]['Global_Sales']
                return 1
            except IndexError:
                return 0

In [179]:
# create new column for rerelease
small_sales_df['rerelease'] = small_sales_df.apply(lambda x: find_rereleases(x['Name'], x['Platform'], small_sales_df), axis=1)

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [181]:
small_sales_df

,Rank,Name,Genre,ESRB_Rating,Platform,Publisher,Developer,Global_Sales,NA_Sales,PAL_Sales,JP_Sales,Other_Sales,Year,metascore,userscore,date,years_ago,month_abbrev,month_num,months_ago,generation_lifespan,rerelease
0,20,Grand Theft Auto V,Action,M,PS3,Rockstar Games,Rockstar North,20.32,6.37,9.85,0.99,3.12,2013,97,8.3,"Sep 17, 2013",7,Sep,9,79,0.975904,1
1,21,Grand Theft Auto V,Action,M,PS4,Rockstar Games,Rockstar North,19.39,6.06,9.71,0.60,3.02,2014,97,8.3,"Nov 18, 2014",6,Nov,11,65,0.142857,2
2,31,Grand Theft Auto: Vice City,Action,M,PS2,Rockstar Games,Rockstar North,16.15,8.41,5.49,0.47,1.78,2002,95,8.8,"Oct 27, 2002",18,Oct,10,210,0.375,0
3,33,Grand Theft Auto V,Action,M,X360,Rockstar Games,Rockstar North,15.86,9.06,5.33,0.06,1.42,2013,97,8.3,"Sep 17, 2013",7,Sep,9,79,0.978947,1
4,41,Call of Duty: Modern Warfare 3,Shooter,M,X360,Activision,Infinity Ward,14.82,9.07,4.29,0.13,1.33,2011,88,3.4,"Nov 08, 2011",9,Nov,11,101,0.747368,0
5,42,Call of Duty: Black Ops,Shooter,M,X360,Activision,Treyarch,14.74,9.76,3.73,0.11,1.14,2010,87,6.4,"Nov 09, 2010",10,Nov,11,113,0.621053,0
6,46,Red Dead Redemption 2,Action-Adventure,M,PS4,Rockstar Games,Rockstar Games,13.94,5.26,6.21,0.21,2.26,2018,97,7.7,"Oct 26, 2018",2,Oct,10,18,0.702381,0
7,47,Call of Duty: Black Ops II,Shooter,M,X360,Activision,Treyarch,13.86,8.27,4.32,0.07,1.20,2012,83,5.0,"Nov 13, 2012",8,Nov,11,89,0.873684,0
8,48,Call of Duty: Black Ops II,Shooter,M,PS3,Activision,Treyarch,13.80,4.99,5.88,0.65,2.28,2012,83,5.4,"Nov 13, 2012",8,Nov,11,89,0.855422,0
9,50,Call of Duty: Modern Warfare 2,Shooter,M,X360,Activision,Infinity Ward,13.53,8.54,3.63,0.08,1.28,2009,94,6.4,"Nov 10, 2009",11,Nov,11,125,0.494737,0
